# 2b: Calculate the data needed for scenario trials

## Plain English summary
This notebook calculates the base pathway statistics for each stroke team. The patients that attended each stroke team are first split into groups for each stroke type. Then a series of tests are performed. The proportion of patients passing each test is recorded, and for certain subgroups of patients passing particular tests, the distribution of times taken at that point in the hospital pathway are measured.

The tests are:
1. Is onset time known?
2. Is onset to arrival within the time limit?
3. Is arrival to scan wtihin the time limit?
4. Is onset to scan within the time limit?
5. Is there enough time left for thrombolysis?
6. Did the patient receive thrombolysis?

And the time distributions measured are:
1. Onset to arrival time for those passing Test 2.
2. Arrival to scan time for those passing Test 3.
3. Scan to treatment time for those passing Test 6.

The values measured are saved for future use.

## Notebook setup:

In [12]:
import pandas as pd
import numpy as np
import pickle
from dataclasses import dataclass
import yaml

import matplotlib.pyplot as plt

import stroke_utilities.scenario as scenario

## Set up paths and filenames

In [13]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    data_read_path: str = './stroke_utilities/data/'
    output_folder = './stroke_utilities/output/'

paths = Paths()

## Import patient data

All patient data:

In [14]:
filename = paths.data_read_path + 'reformatted_data_thrombolysis_decision.csv'
data_loaded = pd.read_csv(filename)

In [15]:
data_loaded.columns

Index(['id', 'stroke_team', 'age', 'male', 'infarction',
       'onset_to_arrival_time', 'precise_onset_known', 'onset_during_sleep',
       'arrive_by_ambulance', 'month', 'year', 'weekday',
       'arrival_time_3_hour_period', 'arrival_to_scan_time', 'thrombolysis',
       'scan_to_thrombolysis_time', 'thrombectomy',
       'arrival_to_thrombectomy_time', 'congestive_heart_failure',
       'hypertension', 'atrial_fibrillation', 'diabetes', 'prior_stroke_tia',
       'afib_antiplatelet', 'afib_anticoagulant', 'prior_disability',
       'stroke_severity', 'nihss_complete', 'nihss_arrival_loc',
       'nihss_arrival_loc_questions', 'nihss_arrival_loc_commands',
       'nihss_arrival_best_gaze', 'nihss_arrival_visual',
       'nihss_arrival_facial_palsy', 'nihss_arrival_motor_arm_left',
       'nihss_arrival_motor_arm_right', 'nihss_arrival_motor_leg_left',
       'nihss_arrival_motor_leg_right', 'nihss_arrival_limb_ataxia',
       'nihss_arrival_sensory', 'nihss_arrival_best_language',


## Process the data


In [16]:
with open('./stroke_utilities/fixed_params.yml') as f:
    fixed_params = yaml.safe_load(f)

In [17]:
fixed_params

{'allowed_onset_to_needle_time_mins': 270,
 'allowed_overrun_for_slow_scan_to_needle_mins': 15,
 'allowed_onset_to_puncture_time_mins': 480,
 'allowed_overrun_for_slow_scan_to_puncture_mins': 15,
 'minutes_left': 15.0,
 'limit_ivt_mins': 240,
 'limit_mt_mins': 360}

In [18]:
# Set up allowed time and over-run for thrombolysis...
allowed_onset_to_needle_time_mins = fixed_params['allowed_onset_to_needle_time_mins']
allowed_overrun_for_slow_scan_to_needle_mins = fixed_params['allowed_overrun_for_slow_scan_to_needle_mins']
# ... and for thrombectomy
allowed_onset_to_puncture_time_mins = fixed_params['allowed_onset_to_puncture_time_mins']
allowed_overrun_for_slow_scan_to_puncture_mins = fixed_params['allowed_overrun_for_slow_scan_to_puncture_mins']
minutes_left = fixed_params['minutes_left']
# Limit for comparing conditions (e.g. is onset to arrival within
# 4hrs?). Separate limits for IVT and MT:
limit_ivt_mins = fixed_params['limit_ivt_mins']
limit_mt_mins = fixed_params['limit_mt_mins']

Combine existing time data to create some new measures:

In [19]:
data_loaded['arrival_to_thrombolysis_time'] = np.sum((
    data_loaded['arrival_to_scan_time'].values,
    data_loaded['scan_to_thrombolysis_time'].values
    ), axis=0)

data_loaded['scan_to_thrombectomy_time'] = np.sum((
    data_loaded['arrival_to_thrombectomy_time'].values,
    -data_loaded['arrival_to_scan_time'].values
    ), axis=0)

# Time left after scan for thrombolysis...
data_loaded['time_left_for_ivt_after_scan_mins'] = np.maximum((
    allowed_onset_to_needle_time_mins -
    (data_loaded['onset_to_arrival_time'] + 
      data_loaded['arrival_to_scan_time'])
    ), -0.0)
# ... and thrombectomy:
data_loaded['time_left_for_mt_after_scan_mins'] = np.maximum((
    allowed_onset_to_puncture_time_mins -
    (data_loaded['onset_to_arrival_time'] + 
      data_loaded['arrival_to_scan_time'])
    ), -0.0)

Measure how many years this data spans. This is used later to calculate the number of admissions per year.

In [7]:
data_years = (data_loaded['year'].max() - data_loaded['year'].min())

data_years

5

## Calculate proportions

The following cell loops over all stroke teams. It finds which patients have each stroke type. Then the subset with each stroke type is passed through the masks and the performance parameters are stored in a pandas Series.

In [8]:
# Copy data
data = data_loaded.copy()
# Split data by stroke team
groups = data.groupby('stroke_team_id') # creates a new object of groups of data

# Store each stroke team's results in this list:
list_of_series = []
for stroke_team, group_df in groups: # each group has an index + dataframe of data
    stroke_type_mask_dict = {
        'lvo': ((group_df['infarction']==1) & 
                (group_df['stroke_severity']>=11)),
        'nlvo': ((group_df['infarction']==1) & 
                 (group_df['stroke_severity']<11)),
        'other': (group_df['infarction']!=1),  # excludes no type given
        'mixed': [True] * len(group_df)
    }
    
    # Split by stroke type:
    for stroke_type in list(stroke_type_mask_dict.keys()):
        group_df_here = group_df[stroke_type_mask_dict[stroke_type]].copy()
        stroke_team_here = stroke_team #+ ': ' + stroke_type
        stroke_type_here = stroke_type
        
        # Main results function:
        group_df_here, group_dict, masks_dict_ivt, masks_dict_mt = (
            scenario.extract_hospital_performance(
                stroke_team_here,
                stroke_type_here,
                group_df_here,
                limit_ivt_mins,
                limit_mt_mins,
                minutes_left
            ))
        # Update admissions, average over the full number of years:
        group_dict['admissions'] = group_dict['admissions'] / data_years
        # Convert output dict into a pandas Series:
        group_series = pd.Series(data=group_dict.values(),
                                 index=group_dict.keys())
        list_of_series.append(group_series)

# Combine all results into one dataframe:
df_all = pd.concat(list_of_series, axis=1)

## Results

In [1]:
df = df_all.T

# Round the values to fewer decimal places:
for column in df.columns:
    if column not in ['stroke_team', 'stroke_type', 'stroke_team_id']:
        df[column] = df[column].astype(float).round(6)

# Save
df.to_csv(f'{paths.output_folder}/hospital_performance.csv', index=False)#, float_format='%.7f')

NameError: name 'df_all' is not defined

In [11]:
# Show data for first hospital
df_all.T.head(4).T

,0,1,2,3
stroke_team_id,1,1,1,1
stroke_type,lvo,nlvo,other,mixed
admissions,79.4,223.8,40.2,343.4
proportion_of_all_with_ivt,0.244332,0.140304,0.0,0.147932
proportion_of_all_with_mt,0.04534,0.001787,0.0,0.011648
proportion_of_mt_with_ivt,0.444444,0.5,NaN,0.45
proportion1_of_all_with_onset_known_ivt,0.513854,0.582663,0.512438,0.558532
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.823529,0.602761,0.757282,0.666319
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.982143,0.959288,0.974359,0.967136
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.951515,0.877984,0.947368,0.906149
